#Scoring

###Prerequisites

The code requires a fixed lmppl-main.zip library (included in repo). This notebook assumes that the archive is located in the same folder.

In [ ]:
! pip install transformers
! pip install torch mxnet-mkl
! pip install sentencepiece

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("rubia.tsv", sep='\t')

data.loc[(data['domain'] == 'nationality') & (data['task_type'] == 'freeform_enemy'), 'task_type'] = 'freeform_full'
data.loc[(data['domain'] == 'class') & (data['task_type'] == 'template_poor'), 'task_type'] = 'template_wealth'
data.loc[(data['domain'] == 'class') & (data['task_type'] == 'template_rich'), 'task_type'] = 'template_wealth'

###Metrics

#####New LM-PPL

In [ ]:
! unzip lmppl-main.zip
! sudo apt-get install python3.10-dev

sh = """
cd lmppl-main
pip install .
"""
with open('script.sh', 'w') as file:
  file.write(sh)

! bash script.sh

In [ ]:
import lmppl

import torch
import transformers
import difflib
import string
from collections import defaultdict

####Functions

The following functions serve to score all correct samples in the dataset with an PPL model

In [ ]:
def ppl_score_model(model, model_name, model_type, data): # device
    vocab = None

    if model_type == 'reccurent':
        scorer = lmppl.LM(model)

        pro_res = []
        anti_res = []

        for i, s in enumerate(data['pro-trope']):
            pro_res.append(scorer.get_perplexity([s])[0])

        for i, s in enumerate(data['anti-trope']):
            anti_res.append(scorer.get_perplexity([s])[0])

    else:
        scorer = lmppl.MaskedLM(model)

        pro_res = []
        anti_res = []

        for i, s in enumerate(data['pro-trope']):
            pro_res.append(scorer.get_perplexity([s])[0])

        for i, s in enumerate(data['anti-trope']):
            anti_res.append(scorer.get_perplexity([s])[0])

    data['ppl-pro-' + model_name] = pro_res
    data['ppl-anti-' + model_name] = anti_res

In [ ]:
def gets_stats(model_name, metric='ppl', to_lists=False):
    domains = []
    tasks = []
    results = []

    for domain in np.unique(data['domain']):
        data_cur = data[(data['domain'] == domain) &
                        (data['task_type'] != 'freeform_gendergap') &
                        (data['task_type'] != 'freeform_family_stereotype') &
                        (data['task_type'] != 'freeform_prof_stereotype') &
                        (data['task_type'] != 'freeform_prof_stereotype')]

        if metric=='ppl':
            cur_bias = len(data_cur[data_cur[metric + '-pro-' + model_name] <
                        data_cur[metric + '-anti-' + model_name]]) / len(data_cur)
        else:
            cur_bias = len(data_cur[data_cur[metric + '-pro-' + model_name] >
                        data_cur[metric + '-anti-' + model_name]]) / len(data_cur)

        if not to_lists:
            print("\n=========================")
            print(domain, "bias: %.3f" %(cur_bias))
        else:
            domains.append(domain)
            tasks.append('all')
            results.append(cur_bias)

        for task_type in np.unique(data[data['domain'] == domain]['task_type']):
            data_cur = data[(data['domain'] == domain) &
                            (data['task_type'] == task_type)]

            if metric=='ppl':
                cur_bias = len(data_cur[data_cur[metric + '-pro-' + model_name] <
                            data_cur[metric + '-anti-' + model_name]]) / len(data_cur)
            else:
                cur_bias = len(data_cur[data_cur[metric + '-pro-' + model_name] >
                            data_cur[metric + '-anti-' + model_name]]) / len(data_cur)

            if not to_lists:
                print('\t', task_type, "bias: %.3f" %(cur_bias))
            else:
                domains.append(domain)
                tasks.append(task_type)
                results.append(cur_bias)

    return domains, tasks, results

In [ ]:
def full_score(model, name, data, device='cuda'):
    ppl_score_model(model, name, data, device)

In [ ]:
def get_stats_all(name):
    domains, tasks, ppls = gets_stats(name, metric='ppl', to_lists=True)

    res = pd.DataFrame()
    res['Domain'] = domains
    res['SubDomain'] = tasks
    res['Model'] = [name] * len(res)
    res['PPL-Score'] = ppls

    return res

In [ ]:
all_res = []

###Test models

####RuGPT Large

1. Large based on GPT2

In [ ]:
%%time
%%capture
model = 'ai-forever/rugpt3large_based_on_gpt2'
full_score(model, "ruGPT-large", 'reccurent', data)

In [ ]:
rugpt_large_res = get_stats_all("ruGPT-large")
rugpt_large_res.to_csv('ruGPT-large.tsv', sep='\t')
all_res.append(rugpt_large_res)
rugpt_large_res

#### RuGPT Base

In [ ]:
%%time
%%capture
model = 'ai-forever/rugpt3medium_based_on_gpt2'
full_score(model, "ruGPT-base", 'reccurent', data)

In [ ]:
rugpt_base_res = get_stats_all("ruGPT-base")
rugpt_base_res.to_csv('ruGPT-base.tsv', sep='\t')
all_res.append(rugpt_base_res)
rugpt_base_res

####XGLM

In [ ]:
%%time
%%capture
model = 'facebook/xglm-564M'
full_score(model, "XGLM", 'reccurent', data)

In [ ]:
xglm_res = get_stats_all("XGLM")
xglm_res.to_csv('XGLM.tsv', sep='\t')
all_res.append(xglm_res)
xglm_res

####mGPT

In [ ]:
%%time
%%capture
model = 'ai-forever/mGPT'
full_score(model, "mGPT", 'reccurent', data)

In [ ]:
mgpt_res = get_stats_all("mGPT")
mgpt_res.to_csv('mGPT.tsv', sep='\t')
all_res.append(mgpt_res)
mgpt_res

#### AI Dungeon

In [ ]:
%%time
%%capture
model = 'imperialwool/ai-dungeon-medium-rus'
full_score(model, "aiDungeon", 'reccurent', data)

In [ ]:
aidungeon_res = get_stats_all("aiDungeon")
aidungeon_res.to_csv('aiDungeon.tsv', sep='\t')
all_res.append(aidungeon_res)
aidungeon_res

###Masked LLMs

#### ruBert base

In [ ]:
%%time
%%capture
model = 'DeepPavlov/rubert-base-cased'
full_score(model, "rubert-base", 'masked', data)

In [ ]:
rubert_base_res = get_stats_all("rubert-base")
rubert_base_res.to_csv('rubert-base.tsv', sep='\t')
all_res.append(rubert_base_res)
rubert_base_res

#### Twitter bert

In [ ]:
%%time
%%capture
model = 'Twitter/twhin-bert-large'
full_score(model, "twhin-bert", 'masked', data)

In [ ]:
twhin_bert_res = get_stats_all("twhin-bert")
twhin_bert_res.to_csv('twhin-bert.tsv', sep='\t')
all_res.append(twhin_bert_res)
twhin_bert_res

#### ruRoberta large

In [ ]:
%%time
%%capture
model = 'ai-forever/ruRoberta-large'
full_score(model, "ruRoberta-large", 'masked', data)

In [ ]:
ruroberta_large_res = get_stats_all("ruRoberta-large")
ruroberta_large_res.to_csv('ruRoberta-large.tsv', sep='\t')
all_res.append(ruroberta_large_res)
ruroberta_large_res

#### ruBert large

In [ ]:
%%time
%%capture
model = 'ai-forever/ruBert-large'
full_score(model, "ruBert-large", 'masked', data)

In [ ]:
rubert_large_res = get_stats_all("ruBert-large")
rubert_large_res.to_csv('ruBert-large.tsv', sep='\t')
all_res.append(rubert_large_res)
rubert_large_res

#### XLM Roberta

In [ ]:
%%time
%%capture
model = 'xlm-roberta-large'
full_score(model, "xlm-roberta-large", 'masked', data)

In [ ]:
xlm_large_res = get_stats_all("xlm-roberta-large")
xlm_large_res.to_csv('xlm-roberta-large.tsv', sep='\t')
all_res.append(xlm_large_res)
xlm_large_res

### All results

In this part we aggregate the results across several models

In [ ]:
table_res = pd.concat(all_res).reset_index()
table_res = table_res.drop(columns=['index'])
data.to_csv('scored_data.tsv', sep="\t")
table_res.to_csv('statistics.tsv', sep="\t")
table_res

In [ ]:
scores = []

for i in range(len(table_res)):
  scores.append(table_res['PPL-Score'][i])

table_res['Score'] = scores
main_res = table_res[['Domain', 'SubDomain', 'Model', 'Score']]
main_res

We also convert them into a more readable format

In [ ]:
model_list = np.unique(main_res['Model'])
model_scores = []

for cur_model in model_list:
    model_res = main_res[main_res['Model'] == cur_model].reset_index()
    scores = model_res['Score']
    domains = model_res['Domain']
    subdomains = model_res['SubDomain']
    model_scores.append(scores)

ans = pd.DataFrame()
ans['Domain'] = domains
ans['Subdomain'] = subdomains

for i in range(len(model_list)):
    ans[model_list[i]] = model_scores[i]

ans.to_csv('model_scores.tsv', sep="\t")
ans

In [ ]:
for i in range(len(model_list)):
    ans[model_list[i]] = ans[model_list[i]].apply(lambda x: round(x * 100, 1))

ans

In [ ]:
ans.to_csv('statistics.tsv', sep='\t')